# Simulation with AR data

In [1]:
%load_ext lab_black

In [2]:
cd ../../

ERROR:root:Cannot parse: 1:4: cd ../../
Traceback (most recent call last):
  File "/Users/adam2392/opt/miniconda3/envs/eztrack/lib/python3.8/site-packages/lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "/Users/adam2392/opt/miniconda3/envs/eztrack/lib/python3.8/site-packages/lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "/Users/adam2392/opt/miniconda3/envs/eztrack/lib/python3.8/site-packages/black.py", line 725, in format_str
    src_node = lib2to3_parse(src_contents.lstrip(), mode.target_versions)
  File "/Users/adam2392/opt/miniconda3/envs/eztrack/lib/python3.8/site-packages/black.py", line 836, in lib2to3_parse
    raise exc from None
black.InvalidInput: Cannot parse: 1:4: cd ../../


/Users/adam2392/Documents/motor-decoding


In [3]:
import pandas as pd
from scipy import stats
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline
import seaborn as sns

import pickle
import time
from tqdm import tqdm

from sklearn.model_selection import train_test_split

# comparative classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

# evaluation classifier
from rerf.rerfClassifier import rerfClassifier

import sys

print(sys.version)
import warnings

warnings.simplefilter("ignore")

%matplotlib inline
%load_ext autoreload
%autoreload 2

3.8.5 | packaged by conda-forge | (default, Jul 22 2020, 17:24:51) 
[Clang 10.0.0 ]


In [4]:
seed = 12345

In [5]:
# length of data sequence
ns = np.array([50, 100, 200, 400, 1000])

# assume you can train on half trials and
# test on half the trials
test_size = 0.5

# number of samples
n_samples = 100

# simulated data parameters
n_trials = 100

# dimensionality of data and noise
data_dim = 3
noise_dim = 3
n_chs = data_dim + noise_dim

# data mean vector
mu = np.zeros((data_dim,))

# permutation strategy
permutation_strategy = "alternate"

# Simulation of AR(1) Data

In [4]:
def generate_ar_samples(cls=0, n_steps=1000, k=1):
    """
    Generate multivariate time-series for 3 channsl of length n_samples
    from a vector AR(1) model. Noise channels are interleaved between
    the vector sequences.

    VAR(1) Model: 
    y(t+1) = Ay(t) + Bu(t) + err(t)

    For now, we ignore exogenous variables, i.e. B = 0, and assume
    uncorrelated errors.
    """

    n_chs = 3
    n_noise_chs = 3

    if cls == 0:
        A = np.array([[
            [  0.5,  0.5*k, 0.7*k],
            [0.3*k, -0.8, -0.15*k],
            [0.5*k,  0.2*k, -0.6]
        ]])
    elif cls == 1:
        A = np.array([[
            [0.5,    0,    0],
            [  0, -0.8,    0],
            [  0,    0, -0.6]
        ]])

    B = np.zeros((n_chs, n_chs))

    A_eigvals = np.linalg.eigvals(A)
    A_spec_rad = np.max(np.abs(A_eigvals))

    # TODO: Figure out how to best set error relative to A matrix
    err_cov = 0.05 * A_spec_rad * np.identity(n_chs)

    var = VARProcess(A, B, err_cov, _params_info={'k_trend': 0})
    y = var.simulate_var(steps=n_steps).T
    assert y.shape == (n_chs, n_steps)

    y_noise = np.random.random((n_noise_chs, n_steps))

    y = np.vstack((
        y[0,:], y_noise[0,:],
        y[1,:], y_noise[1,:],
        y[2,:], y_noise[2,:]
    ))
    
    return y